<hr />

### NoteBookの見方
コード以外の情報 
<span >白/黒での記載は速習Symbol</span>  
<span style="color:red">赤色での記載は補足情報</span>  
<span >コード内で🌟マークがある場所は自分の情報に書き換えが必要</span>

<hr />

<span style="color:red">

# 環境構築
</span>
<span style="color:red">

## 1.Symbol SDKの読み込み </span>

In [ ]:
(script = document.createElement("script")).src = "https://xembook.github.io/nem2-browserify/symbol-sdk-pack-2.0.3.js";
document.getElementsByTagName("head")[0].appendChild(script);

<span style="color:red">

## 2.Symbol用の共通設定 </span>

In [ ]:
NODE = 'https://sym-test-03.opening-line.jp:3001';
sym = require("/node_modules/symbol-sdk");
repo = new sym.RepositoryFactoryHttp(NODE);
txRepo = repo.createTransactionRepository();
mosaicRepo = repo.createMosaicRepository();
accountRepo = repo.createAccountRepository();
(async () => {
  networkType = await repo.getNetworkType().toPromise();
  generationHash = await repo.getGenerationHash().toPromise();
  epochAdjustment = await repo.getEpochAdjustment().toPromise();
})();

function clog(signedTx){
    console.log(NODE + "/transactionStatus/" + signedTx.hash);
    console.log(NODE + "/transactions/confirmed/" + signedTx.hash);
    console.log("https://symbol.fyi/transactions/" + signedTx.hash);
    console.log("https://testnet.symbol.fyi/transactions/" + signedTx.hash);
}

<span style="color:red">

## 3.aliceアカウントのリストア </span>

In [ ]:
alice = sym.Account.createFromPrivateKey(
    "1E9139CC1580B4AED6A1FE110085281D4982ED0D89CE07F3380EB83069B1****", //🌟ここに3章で作成した秘密鍵を入力
    networkType
);

<img width="800" alt="スクリーンショット 2023-01-21 1 39 39" src="https://user-images.githubusercontent.com/47712051/213754095-3cc6bce7-0041-4019-a746-0e18ae5ff684.png">

<img width="800" alt="スクリーンショット 2023-01-21 2 09 33" src="https://user-images.githubusercontent.com/47712051/213761532-f87e074e-9ef4-437c-8b12-d6073351c410.png">

<img width="800" alt="スクリーンショット 2023-01-21 2 09 50" src="https://user-images.githubusercontent.com/47712051/213761554-d9ea8b24-0b86-41c0-bcea-aa076b144fd0.png">

<img width="800" alt="スクリーンショット 2023-01-21 2 10 05" src="https://user-images.githubusercontent.com/47712051/213761569-db5054cb-6bbb-4d83-9fa7-359383d24be3.png">

<img width="800" alt="スクリーンショット 2023-01-21 2 10 19" src="https://user-images.githubusercontent.com/47712051/213761582-dea5cd80-9c09-4d7d-9c9e-42a44b100c96.png">

<img width="800" alt="スクリーンショット 2023-01-21 2 10 34" src="https://user-images.githubusercontent.com/47712051/213761600-00ee6a13-e643-4a94-9361-091869deadb3.png">


<hr />

# 8.ロック

Symbolブロックチェーンにはハッシュロックとシークレットロックの２種類のロック機構があります。  

## 8.1 ハッシュロック

ハッシュロックは後でアナウンスされる予定のトランザクションを事前にハッシュ値で登録しておくことで、
該当トランザクションがアナウンスされた場合に、そのトランザクションをAPIノード上で処理せずにロックさせて、署名が集まってから処理を行うことができます。
アカウントが所有するモザイクを操作できないようにロックするわけではなく、ロックされるのはハッシュ値の対象となるトランザクションとなります。
ハッシュロックにかかる費用は10XYM、有効期限は最大約48時間です。ロックしたトランザクションが承認されれば10XYMは返却されます。

### アグリゲートボンデッドトランザクションの作成


In [ ]:
bob = sym.Account.generateNewAccount(networkType);
tx1 = sym.TransferTransaction.create(
    undefined,
    bob.address,  //Bobへの送信
    [ //1XYM
      new sym.Mosaic(
        new sym.NamespaceId("symbol.xym"),
        sym.UInt64.fromUint(1000000)
      )
    ],
    sym.EmptyMessage, //メッセージ無し
    networkType
);
tx2 = sym.TransferTransaction.create(
    undefined,
    alice.address,  // Aliceへの送信
    [],
    sym.PlainMessage.create('thank you!'), //メッセージ
    networkType
);
aggregateArray = [
    tx1.toAggregate(alice.publicAccount), //Aliceからの送信
    tx2.toAggregate(bob.publicAccount), // Bobからの送信
]
//アグリゲートボンデッドトランザクション
aggregateTx = sym.AggregateTransaction.createBonded(
    sym.Deadline.create(epochAdjustment),
    aggregateArray,
    networkType,
    [],
).setMaxFeeForAggregate(100, 1);
//署名
signedAggregateTx = alice.sign(aggregateTx, generationHash);

tx1,tx2の2つのトランザクションをaggregateArrayで配列にする時に、送信元アカウントの公開鍵を指定します。
公開鍵はアカウントの章を参考に事前にAPIで取得しておきましょう。
配列化されたトランザクションはブロック承認時にその順序で整合性を検証されます。
例えば、tx1でNFTをAliceからBobへ送信した後、tx2でBobからCarolへ同じNFTを送信することは可能ですが、tx2,tx1の順序でアグリゲートトランザクションを通知するとエラーになります。
また、アグリゲートトランザクションの中に1つでも整合性の合わないトランザクションが存在していると、アグリゲートトランザクション全体がエラーとなってチェーンに承認されることはありません。

### ハッシュロックトランザクションの作成と署名、アナウンス

In [ ]:
//ハッシュロックTX作成
hashLockTx = sym.HashLockTransaction.create(
  sym.Deadline.create(epochAdjustment),
    new sym.Mosaic(new sym.NamespaceId("symbol.xym"),sym.UInt64.fromUint(10 * 1000000)), //10xym固定値
    sym.UInt64.fromUint(480), // ロック有効期限
    signedAggregateTx,// このハッシュ値を登録
    networkType
).setMaxFee(100);
//署名
signedLockTx = alice.sign(hashLockTx, generationHash);
//ハッシュロックTXをアナウンス
await txRepo.announce(signedLockTx).toPromise();


In [ ]:
hash = signedLockTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise();
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

### アグリゲートボンデッドトランザクションのアナウンス

エクスプローラーなどで確認した後、ボンデッドトランザクションをネットワークにアナウンスします。


In [ ]:
await txRepo.announceAggregateBonded(signedAggregateTx).toPromise();


In [ ]:
hash = signedAggregateTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Partial).toPromise(); //Partialに変更
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

### 連署
ロックされたトランザクションを指定されたアカウント(Bob)で連署します。

In [ ]:
txInfo = await txRepo.getTransaction(signedAggregateTx.hash,sym.TransactionGroup.Partial).toPromise();
cosignatureTx = sym.CosignatureTransaction.create(txInfo);
signedCosTx = bob.signCosignatureTransaction(cosignatureTx);
await txRepo.announceAggregateBondedCosignature(signedCosTx).toPromise();

In [ ]:
hash = signedAggregateTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise(); //Confirmedに変更
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

### 注意点
ハッシュロックトランザクションは起案者(トランザクションを作成し最初に署名するアカウント)に限らず、誰が作成してアナウンスしても大丈夫ですが、
アグリゲートトランザクションにそのアカウントがsignerとなるトランザクションを含めるようにしてください。
モザイク送信無し＆メッセージ無しのダミートランザクションでも問題ありません（パフォーマンスに影響が出るための仕様とのことです）


## 8.2 シークレットロック・シークレットプルーフ

シークレットロックは事前に共通パスワードを作成しておき、指定モザイクをロックします。
受信者が有効期限内にパスワードの所有を証明することができればロックされたモザイクを受け取ることができる仕組みです。

ここではAliceが1XYMをロックしてBobが解除することで受信する方法を説明します。

まずはAliceとやり取りするBobアカウントを作成します。
ロック解除にBob側からトランザクションをアナウンスする必要があるのでFAUCETで10XYMほど受信しておきます。


In [ ]:
bob = sym.Account.generateNewAccount(networkType);
console.log(bob.address);
//FAUCET URL出力
console.log("https://testnet.symbol.tools/?recipient=" + bob.address.plain() +"&amount=10");

### シークレットロック

ロック・解除にかかわる共通暗号を作成します。


In [ ]:
sha3_256 = require('/node_modules/js-sha3').sha3_256;
random = sym.Crypto.randomBytes(20);
hash = sha3_256.create();
secret = hash.update(random).hex(); //ロック用キーワード
proof = random.toString('hex'); //解除用キーワード
console.log("secret:" + secret);
console.log("proof:" + proof);

###### 出力例
```js
> secret:f260bfb53478f163ee61ee3e5fb7cfcaf7f0b663bc9dd4c537b958d4ce00e240
  proof:7944496ac0f572173c2549baf9ac18f893aab6d0
```

トランザクションを作成・署名・アナウンスします


In [ ]:
lockTx = sym.SecretLockTransaction.create(
    sym.Deadline.create(epochAdjustment),
    new sym.Mosaic(
      new sym.NamespaceId("symbol.xym"),
      sym.UInt64.fromUint(1000000) //1XYM
    ), //ロックするモザイク
    sym.UInt64.fromUint(480), //ロック期間(ブロック数)
    sym.LockHashAlgorithm.Op_Sha3_256, //ロックキーワード生成に使用したアルゴリズム
    secret, //ロック用キーワード
    bob.address, //解除時の転送先:Bob
    networkType
).setMaxFee(100);
signedLockTx = alice.sign(lockTx,generationHash);
await txRepo.announce(signedLockTx).toPromise();


In [ ]:
hash = signedLockTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise(); //Confirmedに変更
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

LockHashAlgorithmは以下の通りです。
```js
{0: 'Op_Sha3_256', 1: 'Op_Hash_160', 2: 'Op_Hash_256'}
```

ロック時に解除先を指定するのでBob以外のアカウントが解除しても転送先（Bob）を変更することはできません。
ロック期間は最長で365日(ブロック数を日換算)までです。

承認されたトランザクションを確認します。


In [ ]:
slRepo = repo.createSecretLockRepository();
res = await slRepo.search({secret:secret}).toPromise();
console.log(res.data[0]);

###### 出力例
```js
> SecretLockInfo
    amount: UInt64 {lower: 1000000, higher: 0}
    compositeHash: "770F65CB0CC0CA17370DE961B2AA5B48B8D86D6DB422171AB00DF34D19DEE2F1"
    endHeight: UInt64 {lower: 323495, higher: 0}
    hashAlgorithm: 0
    mosaicId: MosaicId {id: Id}
    ownerAddress: Address {address: 'TBXUTAX6O6EUVPB6X7OBNX6UUXBMPPAFX7KE5TQ', networkType: 152}
    recipientAddress: Address {address: 'TBTWKXCNROT65CJHEBPL7F6DRHX7UKSUPD7EUGA', networkType: 152}
    recordId: "6260A1D3205E94BEA3D9E3E9"
    secret: "F260BFB53478F163EE61EE3E5FB7CFCAF7F0B663BC9DD4C537B958D4CE00E240"
    status: 0
    version: 1
```
ロックしたAliceがownerAddress、受信予定のBobがrecipientAddressに記録されています。
secret情報が公開されていて、これに対応するproofをBobがネットワークに通知します。


### シークレットプルーフ

解除用キーワードを使用してロック解除します。
Bobは事前に解除用キーワードを入手しておく必要があります。



In [ ]:
proofTx = sym.SecretProofTransaction.create(
    sym.Deadline.create(epochAdjustment),
    sym.LockHashAlgorithm.Op_Sha3_256, //ロック作成に使用したアルゴリズム
    secret, //ロックキーワード
    bob.address, //解除アカウント（受信アカウント）
    proof, //解除用キーワード
    networkType
).setMaxFee(100);
signedProofTx = bob.sign(proofTx,generationHash);
await txRepo.announce(signedProofTx).toPromise();

In [ ]:
hash = signedProofTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise(); //Confirmedに変更
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

SecretProofTransactionにはモザイクの受信量の情報は含まれていません。
ブロック生成時に作成されるレシートで受信量を確認します。
レシートタイプ:LockSecret_Completed でBob宛のレシートを検索してみます。


In [ ]:
receiptRepo = repo.createReceiptRepository();
receiptInfo = await receiptRepo.searchReceipts({
    receiptType:sym.LockSecret_Completed,
    targetAddress:bob.address
}).toPromise();
console.log(receiptInfo.data);


###### 出力例
```js
> data: Array(1)
  >  0: TransactionStatement
        height: UInt64 {lower: 323805, higher: 0}
     >  receipts: Array(1)
          > 0: BalanceChangeReceipt
                amount: UInt64 {lower: 1000000, higher: 0}
            > mosaicId: MosaicId
                  id: Id {lower: 760461000, higher: 981735131}
              targetAddress: Address {address: 'TBTWKXCNROT65CJHEBPL7F6DRHX7UKSUPD7EUGA', networkType: 152}
              type: 8786
```

ReceiptTypeは以下の通りです。

```js
{4685: 'Mosaic_Rental_Fee', 4942: 'Namespace_Rental_Fee', 8515: 'Harvest_Fee', 8776: 'LockHash_Completed', 8786: 'LockSecret_Completed', 9032: 'LockHash_Expired', 9042: 'LockSecret_Expired', 12616: 'LockHash_Created', 12626: 'LockSecret_Created', 16717: 'Mosaic_Expired', 16718: 'Namespace_Expired', 16974: 'Namespace_Deleted', 20803: 'Inflation', 57667: 'Transaction_Group', 61763: 'Address_Alias_Resolution', 62019: 'Mosaic_Alias_Resolution'}
8786: 'LockSecret_Completed' :ロック解除完了
9042: 'LockSecret_Expired'　：ロック期限切れ
```


<span style="color:red">

## 補足</span>

講師に対して

・わずかなxym

・一言メッセージ

を送り代わりにtomatoモザイクをもらう取引を行います。

### アグリゲートボンデッドトランザクションの作成

In [ ]:
target = "TAOP6YRVWUH6VMBAZASNLYHL2SEU7DLALXJN7EY" //講師のアドレス
targetAddress = sym.Address.createFromRawAddress(target)
accountInfo = await accountRepo.getAccountInfo(targetAddress).toPromise();
targetPublicAccount = sym.PublicAccount.createFromPublicKey(
  accountInfo.publicKey,
  networkType
);
tx1 = sym.TransferTransaction.create(
    undefined,
    targetAddress,  //講師へ
    [
      new sym.Mosaic(
        new sym.NamespaceId("symbol.xym"), //XYM
        sym.UInt64.fromUint(1) //数量
      )
    ],
    sym.PlainMessage.create('何か一言'), //🌟一言メッセージ
    networkType
);
tx2 = sym.TransferTransaction.create(
    undefined,
    alice.address,  //自分へ
    [
      new sym.Mosaic(
        new sym.NamespaceId("hossiiii.tomato"), //tomato
        sym.UInt64.fromUint(1) //数量
      )
    ],
    sym.PlainMessage.create('おやつにどうぞ！'),
    networkType
);
aggregateArray = [
    tx1.toAggregate(alice.publicAccount), //自分からtx1を送る
    tx2.toAggregate(targetPublicAccount), // ターゲットからtx2を送る
]
//アグリゲートボンデッドトランザクション
aggregateTx = sym.AggregateTransaction.createBonded(
    sym.Deadline.create(epochAdjustment),
    aggregateArray,
    networkType,
    [],
).setMaxFeeForAggregate(100, 1);
//署名
signedAggregateTx = alice.sign(aggregateTx, generationHash);

### ハッシュロックトランザクションの作成と署名、アナウンス

In [ ]:
//ハッシュロックTX作成
hashLockTx = sym.HashLockTransaction.create(
  sym.Deadline.create(epochAdjustment),
    new sym.Mosaic(new sym.NamespaceId("symbol.xym"),sym.UInt64.fromUint(10 * 1000000)), //10xym固定値
    sym.UInt64.fromUint(480), // ロック有効期限
    signedAggregateTx,// このハッシュ値を登録
    networkType
).setMaxFee(100);
//署名
signedLockTx = alice.sign(hashLockTx, generationHash);
//ハッシュロックTXをアナウンス
await txRepo.announce(signedLockTx).toPromise();

In [ ]:
hash = signedLockTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Confirmed).toPromise(); //Confirmedに変更
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

### アグリゲートボンデッドトランザクションのアナウンス

In [ ]:
await txRepo.announceAggregateBonded(signedAggregateTx).toPromise();

In [ ]:
hash = signedAggregateTx.hash;
tsRepo = repo.createTransactionStatusRepository();
transactionStatus = await tsRepo.getTransactionStatus(hash).toPromise();
console.log(transactionStatus);
txInfo = await txRepo.getTransaction(hash,sym.TransactionGroup.Partial).toPromise(); //Partialに変更
console.log(txInfo);
console.log(`https://testnet.symbol.fyi/transactions/${hash}`) //ブラウザで確認を追加

講師側がpartial状態のトランザクションを確認し、連署を行うことで Unconfirmed => Confirmed状態に遷移し二つの取引が同時に実行されます。

## 8.3 現場で使えるヒント


### 手数料代払い

一般的にブロックチェーンはトランザクション送信に手数料を必要とします。
そのため、ブロックチェーンを利用しようとするユーザは事前に手数料を取引所から入手しておく必要があります。
このユーザが企業である場合はその管理方法も加えてさらにハードルの高い問題となります。
アグリゲートトランザクションを使用することでハッシュロック費用とネットワーク手数料をサービス提供者が代理で負担することができます。

### タイマー送信

シークレットロックは指定ブロック数を経過すると元のアカウントへ払い戻されます。
この原理を利用して、シークレットロックしたアカウントにたいしてロック分の費用をサービス提供者が充足しておけば、
期限が過ぎた後ユーザ側がロック分のトークン所有量が増加することになります。
一方で、期限が過ぎる前にシークレット証明トランザクションをアナウンスすると、送信が完了し、サービス提供者に充当戻るためキャンセル扱いとなります。

### アトミックスワップ
シークレットロックを使用して、他のチェーンとのトークン・モザイクの交換を行うことができます。
他のチェーンではハッシュタイムロックコントラクト(HTLC)と呼ばれているためハッシュロックと間違えないようにご注意ください。


<span style="color:red">

## 参考情報</span>

今回は実施しませんが、ハッシュロックを使った個人間取引を活用し、メタバース上でカイジのエスポワールじゃんけん大会を実施しました。  
また実施してみたいなと思います。  

<img width="800" alt="スクリーンショット 2023-04-21 4 51 10" src="https://user-images.githubusercontent.com/47712051/233473378-1faf0c71-1b40-4aba-bbc3-1aae99848605.png">
